In [1]:
%load_ext autoreload
from scipy import sparse
import numpy as np
from importlib import reload
import mcs

cpx.f     = obj.c;
cpx.sense = 'minimize';
cpx.Aineq = obj.A_ineq;
cpx.bineq = obj.b_ineq;
cpx.Aeq   = obj.A_eq;
cpx.beq   = obj.b_eq;
cpx.lb    = obj.lb;
cpx.ub    = obj.ub;
cpx.ctype = repmat('C',1,size(obj.A_ineq,2));
cpx.ctype(obj.idx_z) = 'B';
obj.indicators.z',...
obj.indicators.inverse',...
obj.indicators.A',1,...
obj.indicators.sense',... 
obj.indicators.b',...
'',...
obj.indicators.type',...
obj.indicators.epln');

cplexmiqp(H, f, Aineq, bineq, Aeq, beq, l, Q, r, sostype, sosind, soswt, lb, ub, ctype, x0, options)

In [2]:
# example setup (default: minimize)
# main problem
c = [-1,0,0,0]
A_ineq = sparse.csr_matrix([[1,1,0,0],[-1,0,0,0]])
b_ineq = [3,-1]
A_eq = sparse.csr_matrix([[0,1,0,0]])
b_eq = [0.6]
lb = [1,0,0,0]
ub = [np.inf, np.inf, 1, 1]
vtype = 'CCBB'
# indicator constraints
ic_binv = [2,3]
ic_A = sparse.csr_matrix([[1,1,0,0],[1,0,0,0]])
ic_b = [0,1.2]
ic_sense = "LE"
ic_indicval = [1,0]
indic_constr = mcs.Indicator_constraints(ic_binv, ic_A, ic_b, ic_sense, ic_indicval)

In [8]:
%autoreload
my_prob = mcs.MILP_LP(  c=c,
                        A_ineq=A_ineq,
                        b_ineq=b_ineq,
                        A_eq=A_eq,
                        b_eq=b_eq,
                        lb=lb,
                        ub=ub,
                        vtype=vtype, 
                        indic_constr=indic_constr,
                        solver='scip')
x, opt_cx, status = my_prob.solve()
# solution.get_status() returns an integer code
print("Solution status = ", str(status))
# the following line prints the corresponding string
print("Solution value  = ", str(round(opt_cx,5)))
sol = {'x'+str(i) : round(x[i],5) for i in range(len(x))}
print(sol)

Solution status =  0
Solution value  =  -2.4
{'x0': 2.4, 'x1': 0.6, 'x2': 0, 'x3': 1}


In [1]:
%load_ext autoreload
import cobra
from importlib import reload
ecc2 = cobra.io.read_sbml_model("ECC2comp.sbml")
ecc3 = cobra.io.read_sbml_model("ECC2comp.sbml")
iml1515 = cobra.io.read_sbml_model("iML1515.sbml")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-09-03


In [9]:
%autoreload
import mcs
from mcs import fva
import cProfile
reload(mcs)
# opt = ecc2.optimize()
# print(opt)
# opt2 = fba(ecc2)
# print(opt2)
# opt3 = cobra.flux_analysis.variability.flux_variability_analysis(iml1515)
# print(opt3)
# cProfile.runctx("mcs.fva(iml1515)",globals(),locals())
opt4 = fva(ecc2, solver='gurobi')
print(opt4)
# cProfile.runctx()

                 minimum      maximum
EX_Biomass      0.000000     2.374995
EX_4CRSOL_ex    0.000000     0.000530
EX_5DRIB_ex     0.000000     0.001594
EX_ac_ex      -10.000000    40.000000
EX_adp_c        0.000000    -0.000000
...                  ...          ...
TALA           -0.468591    45.000000
THD2pp          0.000000  1000.000000
TKT1           -0.468591    45.000000
TKT2           -1.373488    45.000000
TPI          -182.500000    20.000000

[122 rows x 2 columns]
